# Loading Dataframes

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Preprocessing Training set

In [21]:
df['Age'] = df['Age'].fillna(np.nanmean(df['Age']))
df['Fare'] = df['Fare'].fillna(np.nanmean(df['Fare']))

In [5]:
def first(s):          
    k = s[0:1]
    return k
Cabin_class = {}
i=1
df['Cabin'] = df['Cabin'].astype(str)
df['Cabin 1'] = df['Cabin'].apply(first)
for x in df['Cabin 1'].unique():
    Cabin_class[x] = i
    i+=1
print(Cabin_class)
def Cabin_category(s):
    return Cabin_class[s]

{'n': 1, 'C': 2, 'E': 3, 'G': 4, 'D': 5, 'A': 6, 'B': 7, 'F': 8, 'T': 9}


In [6]:
def gender(s):
    if s=='male':
        return 0
    if s=='female':
        return 1

In [7]:
def child_or_adult(i):
    if i>16:
        return 0
    if i<16:
        return 1

In [8]:
embarked_cat = {}
i=1
for x in df['Embarked'].unique():
    embarked_cat[x] = i
    i+=1
def embarkedcat(s):
    return embarked_cat[s]

In [9]:
df['Gender'] = df['Sex'].apply(gender)
df['child'] = df['Age'].apply(child_or_adult)
df['Cabin_class'] = df['Cabin 1'].apply(Cabin_category)
df['Emb_class'] = df['Embarked'].apply(embarkedcat)

In [10]:
df['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [11]:
df_test = df.drop(columns=['Sex','Cabin','Cabin 1','Embarked','Ticket','PassengerId'])

In [20]:
df_test.columns

Index(['Survived', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch', 'Fare', 'Gender',
       'child', 'Cabin_class', 'Emb_class'],
      dtype='object')

In [22]:
df_test_na = df_test.dropna()

In [23]:
df_test_na.shape

(874, 11)

# Preprocessing Testing set

In [46]:
test['Cabin'] = test['Cabin'].astype(str)
test['Cabin 1'] = test['Cabin'].apply(first)

In [47]:
test['Age'] = test['Age'].fillna(np.nanmean(test['Age']))
test['Fare'] = test['Fare'].fillna(np.nanmean(test['Fare']))

In [48]:
test['Gender'] = test['Sex'].apply(gender)
test['child'] = test['Age'].apply(child_or_adult)
test['Cabin_class'] = test['Cabin 1'].apply(Cabin_category)
test['Emb_class'] = test['Embarked'].apply(embarkedcat)

In [75]:
test_pid=test[['PassengerId','Sex']]

In [59]:
test_set = test.drop(columns=['Sex','Cabin','Cabin 1','Embarked','Ticket','PassengerId','Name'])
test_set = test_set.fillna(0)

# Train Test

In [24]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train, y_test = train_test_split(df_test_na.drop(columns=['Survived','Name']),
                                                      df_test_na['Survived'],
                                                      test_size=0.2,
                                                      random_state = 8)

In [25]:
pip install autosklearn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement autosklearn
ERROR: No matching distribution found for autosklearn


In [26]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe
from hpsklearn import random_forest,svc,knn,extra_trees

In [27]:
model = HyperoptEstimator(classifier=any_classifier('clf'), preprocessing=any_preprocessing('pre'), algo=tpe.suggest, trial_timeout=30)

In [18]:
X_train = np.asarray(X_train).astype(np.float32)
y_train=np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_test=np.asarray(y_test).astype(np.float32)

In [29]:
model.fit(X_train, y_train)
# summarize performance
acc = model.score(X_test, y_test)
print("Accuracy: %.3f" % acc)
# summarize the best model
print(model.best_model())

100%|███████████████████████████████████████████████| 10/10 [00:03<00:00,  3.03s/trial, best loss: 0.22857142857142854]
Accuracy: 0.834
{'learner': ExtraTreesClassifier(bootstrap=True, criterion='entropy', max_depth=4,
                     max_features='log2', min_samples_leaf=3, n_estimators=41,
                     n_jobs=1, random_state=3, verbose=False), 'preprocs': (StandardScaler(),), 'ex_preprocs': ()}


# Extra Trees

In [37]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import preprocessing


In [38]:
ExtraTrees = ExtraTreesClassifier(bootstrap=True, criterion='entropy', max_depth=4,
                     max_features='log2', min_samples_leaf=3, n_estimators=41,
                     n_jobs=1, random_state=3, verbose=False)

In [39]:
ExtraTrees.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, criterion='entropy', max_depth=4,
                     max_features='log2', min_samples_leaf=3, n_estimators=41,
                     n_jobs=1, random_state=3, verbose=False)

In [42]:
pred_train= ExtraTrees.predict(X_train)
pred_test= ExtraTrees.predict(X_test)

In [43]:
print('Accuracy via prediction on training set  ', accuracy_score(y_train, pred_train))
print('Accuracy via prediction on training set  ', accuracy_score(y_test, pred_test))

Accuracy via prediction on training set   0.8311874105865522
Accuracy via prediction on training set   0.8342857142857143


In [60]:
test_set_pred = ExtraTrees.predict(test_set)

In [76]:
test_pid

,PassengerId,Sex
0,892,male
1,893,female
2,894,male
3,895,male
4,896,female
...,...,...
413,1305,male
414,1306,female
415,1307,male
416,1308,male


In [79]:
test_set_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [77]:
for i in range(418):
    test_pid['Sex'][i] = test_set_pred[i] 

<ipython-input-77-004c140bfd0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pid['Sex'][i] = test_set_pred[i]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [81]:
test_pid

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [80]:
test_pid.rename(columns = {'Sex':'Survived'}, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [82]:
test_pid.to_csv('gender_submission.csv')

# ANN

In [112]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [105]:
X_train.drop(columns=['Cabin_class'],inplace=True)

In [106]:
X_train.shape

(699, 8)

In [118]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(9, activation='relu', input_shape=[X_train.shape[1],]))
ann.add(tf.keras.layers.Dense(38, activation='relu'))
ann.add(tf.keras.layers.Dense(80, activation='relu'))
ann.add(tf.keras.layers.Dense(256, activation='relu'))
ann.add(tf.keras.layers.Dense(512, activation='relu'))
ann.add(tf.keras.layers.Dense(256, activation='relu'))
ann.add(tf.keras.layers.Dense(30, activation='relu'))
ann.add(tf.keras.layers.Dense(2, activation='softmax'))

In [119]:
ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
ann.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 9)                 81        
_________________________________________________________________
dense_51 (Dense)             (None, 38)                380       
_________________________________________________________________
dense_52 (Dense)             (None, 80)                3120      
_________________________________________________________________
dense_53 (Dense)             (None, 256)               20736     
_________________________________________________________________
dense_54 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_55 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_56 (Dense)             (None, 30)               

In [120]:
ann.fit(X_train, y_train, epochs=400)

Epoch 1/400
22/22 [==============================] - 0s 3ms/step - loss: 0.7246 - accuracy: 0.4807
Epoch 2/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6935 - accuracy: 0.4950
Epoch 3/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6941 - accuracy: 0.5122
Epoch 4/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6945 - accuracy: 0.4864
Epoch 5/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6939 - accuracy: 0.4835
Epoch 6/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6976 - accuracy: 0.4821
Epoch 7/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6947 - accuracy: 0.5136
Epoch 8/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6939 - accuracy: 0.5050
Epoch 9/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.5207
Epoch 10/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6932 - accuracy: 0.5007
Epoch 11/

22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4907
Epoch 84/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.4807
Epoch 85/400
22/22 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.5064
Epoch 86/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6932 - accuracy: 0.4492
Epoch 87/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6932 - accuracy: 0.4950
Epoch 88/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5222
Epoch 89/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5351
Epoch 90/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4635
Epoch 91/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4835
Epoch 92/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4664
Epoch 93/400


22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5351
Epoch 165/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5207
Epoch 166/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4521
Epoch 167/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5422
Epoch 168/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4864
Epoch 169/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5136
Epoch 170/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5136
Epoch 171/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4850
Epoch 172/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5308
Epoch 173/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5050
Epoc

22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4764
Epoch 246/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4993
Epoch 247/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5036
Epoch 248/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5150
Epoch 249/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5107
Epoch 250/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4864
Epoch 251/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4921
Epoch 252/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4979
Epoch 253/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5479
Epoch 254/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5207
Epoc

22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.4979
Epoch 326/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.4735
Epoch 327/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5064
Epoch 328/400
22/22 [==============================] - 0s 3ms/step - loss: 0.6931 - accuracy: 0.5107
Epoch 329/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5050
Epoch 330/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5179
Epoch 331/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5322
Epoch 332/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5222
Epoch 333/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5250
Epoch 334/400
22/22 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.4707
Epoc